In [7]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import argparse
import random
import numpy as np

import os

import data_loader
import model



In [2]:
def init_params(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            init.kaiming_normal_(m.weight, mode='fan_in')
            if m.bias is not None:
                init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            init.constant_(m.weight, 1)
            init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            init.normal_(m.weight, std=1e-3)
            if m.bias is not None:
                init.constant_(m.bias, 0)

In [3]:
trainloader, testloader= data_loader.load_data()

In [4]:
def train(trainloader, net, criterion, optimizer, use_cuda=True):
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        batch_size = inputs.size(0)
        total += batch_size
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*batch_size
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(targets.data).cpu().sum().item()

    return train_loss/total, 100 - 100.*correct/total

In [5]:
def test(testloader, net, criterion, use_cuda=True):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(testloader):
        batch_size = inputs.size(0)
        total += batch_size

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()*batch_size
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(targets.data).cpu().sum().item()

    return test_loss/total, 100 - 100.*correct/total

In [6]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [7]:
lr = 0.01
start_epoch = 1
epoches = 30
criterion = nn.CrossEntropyLoss()
save_folder = 'VGG9'

net = model.VGG('VGG9')

if (torch.cuda.is_available()):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    net.cuda()
    criterion = criterion.cuda()

init_params(net)


VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (14): Conv2d(128, 256, 

In [8]:
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=0.0005)

In [9]:
if not os.path.exists('model_history/' + save_folder):
    os.makedirs('model_history/' + save_folder)


In [10]:
train_loss, train_err = test(trainloader, net, criterion, True)
test_loss, test_err = test(testloader, net, criterion, True)
status = 'e: %d loss: %.5f train_err: %.3f test_top1: %.3f test_loss %.5f \n' % (0, train_loss, train_err, test_err, test_loss)
print(status)

state = {
    'acc': 100 - test_err,
    'epoch': 0,
    'state_dict': net.state_dict()
}
opt_state = {
            'optimizer': optimizer.state_dict()
}
torch.save(state, 'model_history/' + save_folder + '/model_0.t7')
torch.save(opt_state, 'model_history/' + save_folder + '/opt_state_0.t7')

C:\Users\55366\anaconda3\envs\ml\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


e: 0 loss: 2.30273 train_err: 93.913 test_top1: 93.590 test_loss 2.30272 



In [11]:
for epoch in range(start_epoch, 10 + 1):
    loss, train_err = train(trainloader, net, criterion, optimizer, True)
    test_loss, test_err = test(testloader, net, criterion, True)

    status = 'e: %d loss: %.5f train_err: %.3f test_top1: %.3f test_loss %.5f \n' % (epoch, loss, train_err, test_err, test_loss)
    print(status)

    # Save checkpoint.
    acc = 100 - test_err
    state = {
        'acc': acc,
        'epoch': epoch,
        'state_dict': net.state_dict(),
    }
    opt_state = {
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, 'model_history/' + save_folder + '/model_' + str(epoch) + '.t7')
    torch.save(opt_state, 'model_history/' + save_folder + '/opt_state_' + str(epoch) + '.t7')

e: 1 loss: 0.15392 train_err: 4.822 test_top1: 4.150 test_loss 0.12837 

e: 2 loss: 0.08119 train_err: 2.532 test_top1: 5.530 test_loss 0.16206 

e: 3 loss: 0.07172 train_err: 2.198 test_top1: 3.890 test_loss 0.11408 

e: 4 loss: 0.06521 train_err: 1.942 test_top1: 25.930 test_loss 0.88171 

e: 5 loss: 0.06440 train_err: 2.030 test_top1: 1.520 test_loss 0.04847 

e: 6 loss: 0.05857 train_err: 1.822 test_top1: 1.970 test_loss 0.06804 

e: 7 loss: 0.05655 train_err: 1.772 test_top1: 2.900 test_loss 0.10159 

e: 8 loss: 0.05327 train_err: 1.657 test_top1: 2.400 test_loss 0.07430 

e: 9 loss: 0.05261 train_err: 1.578 test_top1: 4.630 test_loss 0.15590 

e: 10 loss: 0.05084 train_err: 1.533 test_top1: 1.570 test_loss 0.04611 

